## 07. 파이토치(PyTorch)의 nn.Embedding()
---

In [ ]:
# 파이토이체어 임베딩 벡터를 사용하는 큰 방법 두 가지
# 1) 임베딩 층(layer)를 만들어서 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
# 2) 미리 사전에 훈련된 임베딩 벡터들을 가져와 사용하는 방법

# 여기서는 전자에 대해 진행한다. nn.Embedding()을 이용

In [4]:
import torch
import torch.nn as nn

### 1. 임베딩 층은 룩업 테이블이다.
---

임베딩 층의 입력으로 사용하기 위해서는 입력 시퀀스의 모든 단어는 정수 인코딩 된 상태여아 한다.  
어떤 단어 -> 단어에 부여된 고유한 정수값 -> 임베딩 층 통과 -> 밀집 벡터(dense vector)  

- 임베딩 층에서는 어떤 일이?  
임베딩 층은 입력 정수에 대해 밀집 벡터로 맵핑하고 이 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련이 된다. 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트가 된다. 그리고 이 밀집 벡터를 임베딩 벡터라고 부른다.  

- 정수는 밀집 벡터 또는 임베딩 벡터로 맵핑한다는 건 어떤 뜻일까?  
특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로 부터 임베딩 벡터 값을 가져오는 룩업 테이블이라고 볼 수 있다. 그리고 이 테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가진다.  

![image](https://wikidocs.net/images/page/33793/lookup_table.PNG)  

위 그림은 단어 great이 정수 인코딩 된 후 테이블로부터 해달 인덱스에 위치한 임베딩 벡터를 꺼내오는 모습을 보여준다.  
임베딩 벡터의 차원: 4  
great의 정수 인코딩: 1918  
great이 사용한 임베딩 벡터의 행 번호: 1918  

이 임베딩 벡터는 모델의 입력이 되고, 역전파 과정에서 단어 great의 임베딩 벡터값이 학습된다.  

- 원-핫 벡터가 아니어도 괜찮은걸까?  
파이토치는 단어를 정수 인덱스로 바꾸고 원-핫 벡터로 한 번 더 바꾸고 나서 임베딩 층의 입력에 사용하는 것이 아니라!  
단어를 정수 인덱스로만 바꾼채로 임베딩 층의 입력으로 사용하더라도 룩업 테이블이 된 결과 즉 임베딩 벡터를 리턴할 수 있다.  

In [3]:
# nn.Embedding()을 사용하지 않고 구현하며 이해해보자
# 우선 임의의 문장으로부터 단어 집합을 만들고 각 단어에 정수를 부여하자

train_data = 'you need to know how to code'
word_set = set(train_data.split())  # 중복을 제거한 단어들의 집합인 단어 집합 생성
vocab = {word: i+2 for i, word in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수를 맵핑

vocab['<unk>'] = 0  # 사전에 없는 단어는 0으로
vocab['<pad>'] = 1  # padding은 1로
print(vocab)

{'you': 2, 'code': 3, 'need': 4, 'to': 5, 'how': 6, 'know': 7, '<unk>': 0, '<pad>': 1}


In [5]:
# 이제 단어 집합의 크기를 행으로 갖는 임베딩 테이블을 구현하자
# 여기서 임베딩 벡터의 차원은 3으로 정했다

embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],      # unk
                               [ 0.0,  0.0,  0.0],      # pad
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [8]:
# 임의의 문장 you need to run 에 대해서 룩업 테이블을 통해 임베딩 벡터들을 가져와보자

sample = 'you need to run'.split()
idxes = []
# 각 단어를 정수로 변환
for word in sample:
    try:
        idxes.append(vocab[word])
    except KeyError:    # 단어 집합에 없는 단어입 경우 <unk>로 대체 해라
        idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes)

print('idxes: ', idxes)

# 룩업 테이블
lookup_result = embedding_table[idxes, :]   # 각 정수를 인덱스로 임베딩 테이블에서 값을 가져오자
print(lookup_result)

idxes:  tensor([2, 4, 5, 0])
tensor([[0.2000, 0.9000, 0.3000],
        [0.2000, 0.1000, 0.8000],
        [0.4000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000]])


### 2. 임베딩 층 사용하기
---

In [9]:
# 임베딩 층을 사용하는 경우를 보자 전처리는 동일하자

train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성.
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [10]:
# nn.Embedding()을 통해 임베딩 테이블을 만들자
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings= len(vocab)   # 임베딩을 할 단어들의 개수 =  집합의 크기
                                , embedding_dim = 3         # 임베딩 할 벡터의 차원
                                , padding_idx = 1           # 패딩의 인덱스
                                )

In [11]:
print(embedding_layer.weight)

Parameter containing:
tensor([[-0.7040, -1.0814,  0.8070],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.3270, -0.9744, -0.1897],
        [-1.2277,  0.1845, -0.3974],
        [-0.1648,  0.7133, -0.7490],
        [-0.6543,  0.0881, -0.7912],
        [-0.7704, -0.4184,  0.6578],
        [ 1.2818, -0.5012,  1.0980]], requires_grad=True)
